# Generative Adversarial Network

## Key Innovations
- Adversarial Training: Introduced a game-theoretic approach where two networks, a generator (G) and a discriminator (D), compete with each other.The generator learns to create data that resembles the real dataset, while the discriminator tries to distinguish between real and generated data.
- Implicit Density Modeling: Unlike traditional generative models, GANs model the data distribution implicitly and do not require an explicit likelihood function.
- Non-Parametric Learning: Avoids restrictive assumptions about the data distribution (e.g., Gaussian).
- Applications: Pioneered numerous applications, including image generation, style transfer, super-resolution, and text-to-image synthesis.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, noise_dim=100, out_channels=1):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z).view(-1, 1, 28, 28)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x.view(-1, 28 * 28))

In [ ]:
# Training GAN
def train_gan():
    # Hyperparameters
    noise_dim = 100
    batch_size = 64
    lr = 0.0002
    epochs = 50

    # Transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    # Dataset and DataLoader
    dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize Generator and Discriminator
    generator = Generator(noise_dim).to('cuda')
    discriminator = Discriminator().to('cuda')

    # Loss and Optimizers
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=lr)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr)

    for epoch in range(epochs):
        for real_images, _ in dataloader:
            # Training Discriminator
            real_images = real_images.to('cuda')
            batch_size = real_images.size(0)

            real_labels = torch.ones(batch_size, 1).to('cuda')
            fake_labels = torch.zeros(batch_size, 1).to('cuda')

            outputs = discriminator(real_images)
            real_loss = criterion(outputs, real_labels)

            z = torch.randn(batch_size, noise_dim).to('cuda')
            fake_images = generator(z)
            outputs = discriminator(fake_images.detach())
            fake_loss = criterion(outputs, fake_labels)

            loss_d = real_loss + fake_loss
            optimizer_d.zero_grad()
            loss_d.backward()
            optimizer_d.step()

            # Training Generator
            z = torch.randn(batch_size, noise_dim).to('cuda')
            fake_images = generator(z)
            outputs = discriminator(fake_images)
            loss_g = criterion(outputs, real_labels)

            optimizer_g.zero_grad()
            loss_g.backward()
            optimizer_g.step()

        print(f"Epoch [{epoch + 1}/{epochs}] | Loss D: {loss_d.item():.4f} | Loss G: {loss_g.item():.4f}")

    # Save generated samples
    with torch.no_grad():
        z = torch.randn(64, noise_dim).to('cuda')
        samples = generator(z).cpu()
        samples = (samples + 1) / 2  # Rescale to [0, 1]

    return samples

# Generate Samples
generated_samples = train_gan()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 54.4MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 2.00MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.14MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Visualization of Generated Samples
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(generated_samples[i].squeeze(), cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.show()